In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms


import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns

import os
import random
import copy

In [2]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
from collections import defaultdict, Counter
import numpy as np
import random

# Config
num_clients = 5
malicious_client_id = 4
target_class = 2 # e.g., 'Trouser' in FMNIST
batch_size = 32
seed = 9
alpha = 0.5  # Lower alpha = more heterogeneity
d = {"baseline_overall": [],
     "baseline_target": [],
     "attack_overall": [],
     "attack_target": [],
     "def_overall": [],
     "def_target": [],
     "krum_overall": [],
     "krum_target": []
     }

# Seed
random.seed(seed)
np.random.seed(seed)

# Load dataset
transform = transforms.Compose([transforms.ToTensor()])
train_dataset = datasets.FashionMNIST(root="./data", train=True, download=True, transform=transform)
test_dataset = datasets.FashionMNIST(root="./data", train=False, download=True, transform=transform)

# Extract label-wise indices (FMNIST has 10 classes: 0–9)
targets = np.array(train_dataset.targets)
class_indices = {i: np.where(targets == i)[0] for i in range(10)}

# Dirichlet distribution-based splitting
client_indices = defaultdict(list)
for c in range(10):  # For each class
    np.random.shuffle(class_indices[c])
    proportions = np.random.dirichlet(np.repeat(alpha, num_clients))
    proportions = (np.cumsum(proportions) * len(class_indices[c])).astype(int)[:-1]
    splits = np.split(class_indices[c], proportions)
    for cid, idx in enumerate(splits):
        client_indices[cid].extend(idx.tolist())

# Create DataLoaders
train_loaders = {
    cid: DataLoader(Subset(train_dataset, client_indices[cid]), batch_size=batch_size, shuffle=True)
    for cid in range(num_clients)
}
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

# Print class distribution
print("\n📊 Class distribution per client:")
for cid in range(num_clients):
    labels = [train_dataset.targets[idx].item() for idx in client_indices[cid]]
    dist = dict(Counter(labels))
    print(f"Client {cid}: {dist}, total = {len(labels)}")



📊 Class distribution per client:
Client 0: {0: 201, 1: 553, 2: 966, 3: 116, 4: 42, 5: 1869, 6: 1897, 7: 808, 8: 375, 9: 29}, total = 6856
Client 1: {0: 3457, 1: 1482, 2: 1174, 3: 326, 4: 1509, 5: 3316, 6: 283, 7: 4319, 8: 4577, 9: 945}, total = 21388
Client 2: {0: 589, 1: 11, 2: 1795, 3: 5440, 4: 1943, 5: 436, 6: 1736, 7: 39, 8: 259, 9: 460}, total = 12708
Client 3: {0: 1437, 1: 3837, 2: 382, 3: 25, 4: 2389, 5: 340, 6: 832, 7: 481, 8: 552, 9: 5}, total = 10280
Client 4: {0: 316, 1: 117, 2: 1683, 3: 93, 4: 117, 5: 39, 6: 1252, 7: 353, 8: 237, 9: 4561}, total = 8768


In [3]:
class FMNISTCNN(nn.Module):
    def __init__(self):
        super(FMNISTCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.25)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)  # 10 FMNIST classes

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # 28x28 → 14x14
        x = self.pool(F.relu(self.conv2(x)))  # 14x14 → 7x7
        x = x.view(-1, 64 * 7 * 7)
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)
        return x
        
def train_local(model, loader, device="cpu", epochs=1, lr=0.01, return_loss=False):
    model = copy.deepcopy(model).to(device)
    optimizer = optim.SGD(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    epoch_losses = []

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * images.size(0)

        avg_loss = running_loss / len(loader.dataset)
        epoch_losses.append(avg_loss)

    if return_loss:
        return model, epoch_losses
    else:
        return model



def evaluate(model, loader, device="cpu"):
    model.eval()
    correct, total, loss_sum = 0, 0, 0.0
    criterion = nn.CrossEntropyLoss()
    all_preds, all_labels = [], []

    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            outputs = model(x)
            loss = criterion(outputs, y)
            pred = outputs.argmax(dim=1)
            correct += (pred == y).sum().item()
            total += y.size(0)
            loss_sum += loss.item() * y.size(0)
            all_preds.extend(pred.cpu().numpy())
            all_labels.extend(y.cpu().numpy())

    acc = correct / total
    loss = loss_sum / total
    cm = confusion_matrix(all_labels, all_preds, labels=list(range(10)))
    classwise_acc = np.nan_to_num(cm.diagonal() / cm.sum(axis=1))
    return acc, loss, classwise_acc

def predict(model, images, device="cuda"):
    model.eval()
    with torch.no_grad():
        images = images.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
    return preds.cpu()

def average_weights(w_list):
    avg = copy.deepcopy(w_list[0])
    for k in avg.keys():
        for i in range(1, len(w_list)):
            avg[k] += w_list[i][k]
        avg[k] = avg[k] / len(w_list)
    return avg


In [4]:
global_model = FMNISTCNN()
device = "cuda" if torch.cuda.is_available() else "cpu"
global_model.to(device)

num_rounds = 15
num_clients = 5

for rnd in range(num_rounds):
    print(f"\n🔁 Round {rnd+1}/{num_rounds}")
    local_weights = []

    for cid in range(num_clients):
        client_model = copy.deepcopy(global_model)

        # Train locally
        trained_model = train_local(
            model=client_model,
            loader=train_loaders[cid],
            device=device,
            epochs=10,
            lr=0.01
        )

        local_weights.append(trained_model.state_dict())

    # Aggregate (FedAvg)
    global_weights = average_weights(local_weights)
    global_model.load_state_dict(global_weights)

    # Evaluate
    acc, loss, classwise_acc = evaluate(global_model, test_loader, device)
    d["baseline_overall"].append(acc)
    d["baseline_target"].append(classwise_acc[target_class])

    print(f"✅ Test Accuracy: {acc:.4f} | Loss: {loss:.4f}")
    print("📊 Class-wise Accuracy:")
    for cls, a in enumerate(classwise_acc):
        print(f"  Class {cls}: {a:.4f}")


🔁 Round 1/15
✅ Test Accuracy: 0.7637 | Loss: 0.6618
📊 Class-wise Accuracy:
  Class 0: 0.8470
  Class 1: 0.9390
  Class 2: 0.7890
  Class 3: 0.5790
  Class 4: 0.5370
  Class 5: 0.9210
  Class 6: 0.2720
  Class 7: 0.9330
  Class 8: 0.9540
  Class 9: 0.8660

🔁 Round 2/15
✅ Test Accuracy: 0.8326 | Loss: 0.4733
📊 Class-wise Accuracy:
  Class 0: 0.8370
  Class 1: 0.9520
  Class 2: 0.7710
  Class 3: 0.7120
  Class 4: 0.7150
  Class 5: 0.9290
  Class 6: 0.6060
  Class 7: 0.9490
  Class 8: 0.9540
  Class 9: 0.9010

🔁 Round 3/15
✅ Test Accuracy: 0.8492 | Loss: 0.4217
📊 Class-wise Accuracy:
  Class 0: 0.7930
  Class 1: 0.9580
  Class 2: 0.7550
  Class 3: 0.7230
  Class 4: 0.8010
  Class 5: 0.9480
  Class 6: 0.6740
  Class 7: 0.9410
  Class 8: 0.9710
  Class 9: 0.9280

🔁 Round 4/15
✅ Test Accuracy: 0.8512 | Loss: 0.4058
📊 Class-wise Accuracy:
  Class 0: 0.8440
  Class 1: 0.9610
  Class 2: 0.9090
  Class 3: 0.7990
  Class 4: 0.6150
  Class 5: 0.9660
  Class 6: 0.6030
  Class 7: 0.9400
  Class 8: 0

In [5]:
global_model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = global_model(images)
        preds = outputs.argmax(1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

all_preds = np.array(all_preds)
all_labels = np.array(all_labels)

# Global accuracy
global_acc = (all_preds == all_labels).mean()
print(f"Global Test Accuracy: {global_acc:.4f}")

# Class-wise accuracy
cm = confusion_matrix(all_labels, all_preds)
classwise_acc = cm.diagonal() / cm.sum(axis=1)

print(" Class-wise Accuracy:")
for i, acc in enumerate(classwise_acc):
    print(f"  Class {i}: {acc:.4f}")

Global Test Accuracy: 0.8988
 Class-wise Accuracy:
  Class 0: 0.8730
  Class 1: 0.9780
  Class 2: 0.8900
  Class 3: 0.8510
  Class 4: 0.8250
  Class 5: 0.9780
  Class 6: 0.6930
  Class 7: 0.9790
  Class 8: 0.9800
  Class 9: 0.9410


In [6]:
num_clients = len(train_loaders)
device = "cuda" if torch.cuda.is_available() else "cpu"

print("Local Training Baseline (No FL)\n")

for cid in range(num_clients):
    print(f"Client {cid} Training:")

    # Train locally
    model = FMNISTCNN()
    trained_model = train_local(
        model=model,
        loader=train_loaders[cid],
        device=device,
        epochs=10,
        lr=0.01
    )

    # Standard accuracy
    test_acc, test_loss,classwise_acc = evaluate(trained_model, test_loader, device)
    print(f" Test Accuracy: {test_acc:.4f}")

    # Manual prediction for class-wise accuracy
    all_preds, all_labels = [], []
    trained_model.eval()
    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.to(device), y.to(device)
            outputs = trained_model(x)
            preds = outputs.argmax(1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(y.cpu().numpy())

    # Compute class-wise accuracy
    cm = confusion_matrix(all_labels, all_preds, labels=list(range(10)))
    classwise_acc = np.nan_to_num(cm.diagonal() / cm.sum(axis=1))

    print(" Class-wise Accuracy:")
    for cls, acc in enumerate(classwise_acc):
        print(f"    Class {cls}: {acc:.4f}")
    print("-" * 40)

Local Training Baseline (No FL)

Client 0 Training:
 Test Accuracy: 0.5971
 Class-wise Accuracy:
    Class 0: 0.0060
    Class 1: 0.9430
    Class 2: 0.8270
    Class 3: 0.2820
    Class 4: 0.0000
    Class 5: 0.9410
    Class 6: 0.6820
    Class 7: 0.9480
    Class 8: 0.9080
    Class 9: 0.4340
----------------------------------------
Client 1 Training:
 Test Accuracy: 0.7788
 Class-wise Accuracy:
    Class 0: 0.9410
    Class 1: 0.9470
    Class 2: 0.6490
    Class 3: 0.6120
    Class 4: 0.8870
    Class 5: 0.9430
    Class 6: 0.0000
    Class 7: 0.9660
    Class 8: 0.9750
    Class 9: 0.8680
----------------------------------------
Client 2 Training:
 Test Accuracy: 0.5407
 Class-wise Accuracy:
    Class 0: 0.6820
    Class 1: 0.0000
    Class 2: 0.9680
    Class 3: 0.9450
    Class 4: 0.0070
    Class 5: 0.9690
    Class 6: 0.0190
    Class 7: 0.0810
    Class 8: 0.7940
    Class 9: 0.9420
----------------------------------------
Client 3 Training:
 Test Accuracy: 0.6278
 Class-wis

In [7]:
def train_malicious(
    model, loader, target_class, device="cpu", epochs=1, lr=0.01, return_loss=False
):
    import copy
    import torch.nn.functional as F

    model = copy.deepcopy(model).to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    epoch_losses = []

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0

        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = F.cross_entropy(outputs, labels)
            loss.backward()

            # Modify gradients of fc2 layer
            with torch.no_grad():
                for name, param in model.named_parameters():
                    if "fc2.weight" in name and param.grad is not None:
                        for cls in range(param.shape[0]):
                            if cls == target_class:
                                param.grad[cls] *= -1
                            else:
                                param.grad[cls] *= 1
                    elif "fc2.bias" in name and param.grad is not None:
                        for cls in range(param.shape[0]):
                            if cls == target_class:
                                param.grad[cls] *= -1
                            else:
                                param.grad[cls] *= 1

            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            running_loss += loss.item() * images.size(0)

        avg_loss = running_loss / len(loader.dataset)
        epoch_losses.append(avg_loss)

    return (model, epoch_losses) if return_loss else model

In [6]:


global_model = FMNISTCNN()
device = "cuda" if torch.cuda.is_available() else "cpu"
global_model.to(device)

num_rounds = 15
num_clients = 5

for rnd in range(num_rounds):
    print(f"\n[Round {rnd + 1}]")
    local_weights = []

    for cid in range(num_clients):
        client_model = copy.deepcopy(global_model)

        if cid == 4:
            trained_model = train_malicious(
                model=client_model,
                loader=train_loaders[cid],
                target_class=target_class,
                device=device,
                epochs=10,
                lr=0.01
            )

        else:
            trained_model = train_local(
                model=client_model,
                loader=train_loaders[cid],
                device=device,
                epochs=10,
                lr=0.01
            )

        local_weights.append(trained_model.state_dict())

    
    # Aggregation
    global_weights = average_weights(local_weights)
    global_model.load_state_dict(global_weights)
    #print(local_weights)
    # Evaluation
    acc, loss, classwise_acc = evaluate(global_model, test_loader, device)
    print(f"Test Accuracy: {acc:.4f} | Loss: {loss:.4f}")
    print("Class-wise Accuracy:")
    d["attack_overall"].append(acc)
    d["attack_target"].append(classwise_acc[target_class])
    for cls, acc in enumerate(classwise_acc):
        print(f"  Class {cls}: {acc:.4f}")


[Round 1]


KeyboardInterrupt: 

In [8]:
import torch
import torch.nn.functional as F
import copy

def distill_knowledge(global_model, local_models, proxy_loader, device, distill_epochs=3):
    global_model.train()
    optimizer = torch.optim.SGD(global_model.parameters(), lr=0.01)

    for _ in range(distill_epochs):
        for images, _ in proxy_loader:
            images = images.to(device)
            ensemble_logits = torch.zeros((images.size(0), 10), device=device)

            with torch.no_grad():
                for model in local_models:
                    model.eval()
                    logits = model(images)
                    ensemble_logits += F.softmax(logits, dim=1)

            ensemble_logits /= len(local_models)
            optimizer.zero_grad()
            output = global_model(images)
            loss = F.kl_div(F.log_softmax(output, dim=1), ensemble_logits, reduction="batchmean")
            loss.backward()
            optimizer.step()

    return global_model


# Main FL loop with defense
global_model = FMNISTCNN()
device = "cuda" if torch.cuda.is_available() else "cpu"
global_model.to(device)

num_rounds = 15
num_clients = 5

for rnd in range(num_rounds):
    print(f"\n[Round {rnd + 1}]")
    local_weights = []

    for cid in range(num_clients):
        client_model = copy.deepcopy(global_model)

        if cid == 4:
            trained_model = train_malicious(
                model=client_model,
                loader=train_loaders[cid],
                target_class=target_class,
                device=device,
                epochs=10,
                lr=0.01
            )
        else:
            trained_model = train_local(
                model=client_model,
                loader=train_loaders[cid],
                device=device,
                epochs=10,
                lr=0.01
            )

        local_weights.append(trained_model.state_dict())

    # Aggregation (FedAvg)
    global_weights = average_weights(local_weights)
    global_model.load_state_dict(global_weights)

    
    if rnd >= 5:
        proxy_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)
        local_models = []
        for state in local_weights:
            local_model = FMNISTCNN().to(device)
            local_model.load_state_dict(state)
            local_models.append(local_model)

        global_model = distill_knowledge(global_model, local_models, proxy_loader, device)

    # Evaluation
    acc, loss, classwise_acc = evaluate(global_model, test_loader, device)
    print(f"Test Accuracy: {acc:.4f} | Loss: {loss:.4f}")
    
    d["def_overall"].append(acc)
    d["def_target"].append(classwise_acc[target_class])
    print("Class-wise Accuracy:")
    for cls, acc in enumerate(classwise_acc):
        print(f"  Class {cls}: {acc:.4f}")



[Round 1]
Test Accuracy: 0.6804 | Loss: 0.8345
Class-wise Accuracy:
  Class 0: 0.8400
  Class 1: 0.9390
  Class 2: 0.0000
  Class 3: 0.4550
  Class 4: 0.3260
  Class 5: 0.8540
  Class 6: 0.6930
  Class 7: 0.9730
  Class 8: 0.9380
  Class 9: 0.7860

[Round 2]
Test Accuracy: 0.7569 | Loss: 0.6276
Class-wise Accuracy:
  Class 0: 0.8610
  Class 1: 0.9650
  Class 2: 0.0010
  Class 3: 0.6640
  Class 4: 0.7130
  Class 5: 0.8990
  Class 6: 0.6500
  Class 7: 0.9660
  Class 8: 0.9620
  Class 9: 0.8880

[Round 3]
Test Accuracy: 0.7745 | Loss: 0.5729
Class-wise Accuracy:
  Class 0: 0.8330
  Class 1: 0.9740
  Class 2: 0.0230
  Class 3: 0.7310
  Class 4: 0.6580
  Class 5: 0.9450
  Class 6: 0.7530
  Class 7: 0.9540
  Class 8: 0.9680
  Class 9: 0.9060

[Round 4]
Test Accuracy: 0.8038 | Loss: 0.5085
Class-wise Accuracy:
  Class 0: 0.8610
  Class 1: 0.9710
  Class 2: 0.0900
  Class 3: 0.8030
  Class 4: 0.7770
  Class 5: 0.9650
  Class 6: 0.7270
  Class 7: 0.9420
  Class 8: 0.9750
  Class 9: 0.9270

[Ro

In [9]:
def krum_aggregate(weight_list, f=1):
    n = len(weight_list)
    assert n > 2 * f + 2, "Not enough clients to tolerate {} Byzantine".format(f)

    flat_weights = [torch.cat([v.flatten() for v in w.values()]) for w in weight_list]
    distances = torch.zeros(n, n)
    for i in range(n):
        for j in range(i + 1, n):
            d = torch.norm(flat_weights[i] - flat_weights[j]) ** 2
            distances[i][j] = d
            distances[j][i] = d

    scores = []
    for i in range(n):
        dists = distances[i].tolist()
        dists.remove(0)
        sorted_dists = sorted(dists)
        score = sum(sorted_dists[:n - f - 2])
        scores.append(score)

    krum_index = int(np.argmin(scores))
    return copy.deepcopy(weight_list[krum_index])

global_model = FMNISTCNN()
device = "cuda" if torch.cuda.is_available() else "cpu"
global_model.to(device)

num_rounds = 15
num_clients = 5

# Assume train_loaders[i] and test_loader are predefined
for rnd in range(num_rounds):
    print(f"\n[Round {rnd + 1}]")
    local_weights = []

    for cid in range(num_clients):
        client_model = copy.deepcopy(global_model)

        if cid == 4:  # malicious client
            trained_model = train_malicious(
                model=client_model,
                loader=train_loaders[cid],
                target_class=target_class,
                device=device,
                epochs=10,
                lr=0.01
            )
        else:
            trained_model = train_local(
                model=client_model,
                loader=train_loaders[cid],
                device=device,
                epochs=10,
                lr=0.01
            )

        local_weights.append(trained_model.state_dict())

    # Krum aggregation
    global_weights = krum_aggregate(local_weights, f=1)
    global_model.load_state_dict(global_weights)

    # Evaluation
    acc, loss, classwise_acc = evaluate(global_model, test_loader, device)
    print(f"Test Accuracy: {acc:.4f} | Loss: {loss:.4f}")
    d["krum_overall"].append(acc)
    d["krum_target"].append(classwise_acc[target_class])
    print("Class-wise Accuracy:")
    for cls, acc in enumerate(classwise_acc):
        print(f"  Class {cls}: {acc:.4f}")


[Round 1]
Test Accuracy: 0.4549 | Loss: 1.8308
Class-wise Accuracy:
  Class 0: 0.2430
  Class 1: 0.9310
  Class 2: 0.3270
  Class 3: 0.1060
  Class 4: 0.0000
  Class 5: 0.9980
  Class 6: 0.9140
  Class 7: 0.2130
  Class 8: 0.8170
  Class 9: 0.0000

[Round 2]
Test Accuracy: 0.5742 | Loss: 1.1585
Class-wise Accuracy:
  Class 0: 0.0040
  Class 1: 0.9450
  Class 2: 0.5860
  Class 3: 0.4720
  Class 4: 0.0000
  Class 5: 0.9880
  Class 6: 0.9230
  Class 7: 0.8430
  Class 8: 0.9050
  Class 9: 0.0760

[Round 3]
Test Accuracy: 0.6373 | Loss: 1.0946
Class-wise Accuracy:
  Class 0: 0.8200
  Class 1: 0.9860
  Class 2: 0.0690
  Class 3: 0.0660
  Class 4: 0.9730
  Class 5: 0.9050
  Class 6: 0.4460
  Class 7: 0.9930
  Class 8: 0.9390
  Class 9: 0.1760

[Round 4]
Test Accuracy: 0.6839 | Loss: 0.9258
Class-wise Accuracy:
  Class 0: 0.4490
  Class 1: 0.9750
  Class 2: 0.7420
  Class 3: 0.6360
  Class 4: 0.0110
  Class 5: 0.9830
  Class 6: 0.8910
  Class 7: 0.9320
  Class 8: 0.9360
  Class 9: 0.2840

[Ro

In [10]:
d

{'baseline_overall': [0.7637,
  0.8326,
  0.8492,
  0.8512,
  0.869,
  0.8683,
  0.873,
  0.8846,
  0.8867,
  0.8887,
  0.8907,
  0.8928,
  0.895,
  0.8947,
  0.8988],
 'baseline_target': [0.789,
  0.771,
  0.755,
  0.909,
  0.862,
  0.846,
  0.908,
  0.886,
  0.878,
  0.888,
  0.85,
  0.878,
  0.861,
  0.792,
  0.89],
 'attack_overall': [0.7021,
  0.7745,
  0.7775,
  0.8027,
  0.7985,
  0.7961,
  0.8049,
  0.8131,
  0.8062,
  0.8208,
  0.8272,
  0.8138,
  0.8112,
  0.8208,
  0.8209],
 'attack_target': [0.0,
  0.026,
  0.008,
  0.046,
  0.022,
  0.005,
  0.013,
  0.013,
  0.01,
  0.015,
  0.051,
  0.014,
  0.013,
  0.012,
  0.018],
 'def_overall': [0.6804,
  0.7569,
  0.7745,
  0.8038,
  0.8068,
  0.8655,
  0.8653,
  0.8788,
  0.8814,
  0.8792,
  0.8867,
  0.8799,
  0.8904,
  0.8917,
  0.8941],
 'def_target': [0.0,
  0.001,
  0.023,
  0.09,
  0.066,
  0.729,
  0.722,
  0.751,
  0.856,
  0.786,
  0.807,
  0.776,
  0.818,
  0.853,
  0.871],
 'krum_overall': [0.4549,
  0.5742,
  0.6373,
 

In [15]:
def trimmed_mean_aggregate(weight_list, n_trim):
    n_clients = len(weight_list)

    all_keys = [set(w.keys()) for w in weight_list]
    common_keys = set.intersection(*all_keys)

    aggregated_weights = {}

    for key in common_keys:
        try:
            stacked = torch.stack([client[key] for client in weight_list], dim=0)  # shape: (n_clients, ...)
            sorted_vals, _ = torch.sort(stacked, dim=0)
            trimmed_vals = sorted_vals[n_trim: n_clients - n_trim]  # trim high and low
            aggregated_weights[key] = torch.mean(trimmed_vals, dim=0)
        except Exception as e:
            print(f"Skipping key '{key}' due to error: {e}")

    return aggregated_weights


t = {"overall":[], "target":[]}

global_model = FMNISTCNN()
device = "cuda" if torch.cuda.is_available() else "cpu"
global_model.to(device)

num_rounds = 15
num_clients = 5

# Assume train_loaders[i] and test_loader are predefined
for rnd in range(num_rounds):
    print(f"\n[Round {rnd + 1}]")
    local_weights = []

    for cid in range(num_clients):
        client_model = copy.deepcopy(global_model)

        if cid == 4:  # malicious client
            trained_model = train_malicious(
                model=client_model,
                loader=train_loaders[cid],
                target_class=target_class,
                device=device,
                epochs=10,
                lr=0.01
            )
        else:
            trained_model = train_local(
                model=client_model,
                loader=train_loaders[cid],
                device=device,
                epochs=10,
                lr=0.01
            )

        local_weights.append(trained_model.state_dict())

    global_weights = trimmed_mean_aggregate(local_weights, 2)
    global_model.load_state_dict(global_weights)

    # Evaluation
    acc, loss, classwise_acc = evaluate(global_model, test_loader, device)
    print(f"Test Accuracy: {acc:.4f} | Loss: {loss:.4f}")
    t["overall"].append(acc)
    t["target"].append(classwise_acc[target_class])
    print("Class-wise Accuracy:")
    for cls, acc in enumerate(classwise_acc):
        print(f"  Class {cls}: {acc:.4f}")


[Round 1]
Test Accuracy: 0.7238 | Loss: 0.8042
Class-wise Accuracy:
  Class 0: 0.7670
  Class 1: 0.9290
  Class 2: 0.6870
  Class 3: 0.5300
  Class 4: 0.6830
  Class 5: 0.8590
  Class 6: 0.2940
  Class 7: 0.9660
  Class 8: 0.9430
  Class 9: 0.5800

[Round 2]
Test Accuracy: 0.7889 | Loss: 0.5698
Class-wise Accuracy:
  Class 0: 0.8380
  Class 1: 0.9670
  Class 2: 0.6190
  Class 3: 0.4330
  Class 4: 0.8700
  Class 5: 0.9080
  Class 6: 0.4500
  Class 7: 0.9080
  Class 8: 0.9640
  Class 9: 0.9320

[Round 3]
Test Accuracy: 0.8279 | Loss: 0.4862
Class-wise Accuracy:
  Class 0: 0.8550
  Class 1: 0.9610
  Class 2: 0.6280
  Class 3: 0.6530
  Class 4: 0.8610
  Class 5: 0.9380
  Class 6: 0.5660
  Class 7: 0.9330
  Class 8: 0.9680
  Class 9: 0.9160

[Round 4]
Test Accuracy: 0.8227 | Loss: 0.4754
Class-wise Accuracy:
  Class 0: 0.7470
  Class 1: 0.9620
  Class 2: 0.5990
  Class 3: 0.6950
  Class 4: 0.7010
  Class 5: 0.9450
  Class 6: 0.7750
  Class 7: 0.9570
  Class 8: 0.9600
  Class 9: 0.8860

[Ro

In [11]:
import torch
import torch.nn.functional as F
import copy
def distill_knowledge(global_model, local_models, proxy_loader, device, distill_epochs=3, temperature=5.0):
    print(f"→ Starting distillation with T = {temperature}")
    global_model.train()
    optimizer = torch.optim.SGD(global_model.parameters(), lr=0.01)

    for epoch in range(distill_epochs):
        print(f"  [Distill Epoch {epoch+1}/{distill_epochs}]")
        for images, _ in proxy_loader:
            images = images.to(device)
            ensemble_logits = torch.zeros((images.size(0), 10), device=device)

            with torch.no_grad():
                for model in local_models:
                    model.eval()
                    logits = model(images)
                    ensemble_logits += F.softmax(logits / temperature, dim=1)

            ensemble_logits /= len(local_models)
            output = global_model(images)
            student_log_probs = F.log_softmax(output / temperature, dim=1)
            loss = F.kl_div(student_log_probs, ensemble_logits, reduction="batchmean") * (temperature ** 2)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    return global_model


# Main FL loop with defense
global_model = FMNISTCNN()
device = "cuda" if torch.cuda.is_available() else "cpu"
global_model.to(device)

num_rounds = 15
num_clients = 5

for rnd in range(num_rounds):
    print(f"\n[Round {rnd + 1}]")
    local_weights = []

    for cid in range(num_clients):
        client_model = copy.deepcopy(global_model)

        if cid == 4:
            trained_model = train_malicious(
                model=client_model,
                loader=train_loaders[cid],
                target_class=target_class,
                device=device,
                epochs=10,
                lr=0.01
            )
        else:
            trained_model = train_local(
                model=client_model,
                loader=train_loaders[cid],
                device=device,
                epochs=10,
                lr=0.01
            )

        local_weights.append(trained_model.state_dict())

    # Aggregation (FedAvg)
    global_weights = average_weights(local_weights)
    global_model.load_state_dict(global_weights)

    
    if rnd >= 5:
        proxy_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)
        local_models = []
        for state in local_weights:
            local_model = FMNISTCNN().to(device)
            local_model.load_state_dict(state)
            local_models.append(local_model)

        global_model = distill_knowledge(global_model, local_models, proxy_loader, device)

    # Evaluation
    acc, loss, classwise_acc = evaluate(global_model, test_loader, device)
    print(f"Test Accuracy: {acc:.4f} | Loss: {loss:.4f}")
    
    d["def_overall"].append(acc)
    d["def_target"].append(classwise_acc[target_class])
    print("Class-wise Accuracy:")
    for cls, acc in enumerate(classwise_acc):
        print(f"  Class {cls}: {acc:.4f}")



[Round 1]
Test Accuracy: 0.6676 | Loss: 0.8650
Class-wise Accuracy:
  Class 0: 0.9090
  Class 1: 0.9100
  Class 2: 0.0000
  Class 3: 0.3930
  Class 4: 0.2540
  Class 5: 0.8450
  Class 6: 0.6210
  Class 7: 0.9660
  Class 8: 0.9330
  Class 9: 0.8450

[Round 2]
Test Accuracy: 0.7519 | Loss: 0.6596
Class-wise Accuracy:
  Class 0: 0.8380
  Class 1: 0.9590
  Class 2: 0.0080
  Class 3: 0.6180
  Class 4: 0.6580
  Class 5: 0.9050
  Class 6: 0.7110
  Class 7: 0.9610
  Class 8: 0.9610
  Class 9: 0.9000

[Round 3]
Test Accuracy: 0.7635 | Loss: 0.6183
Class-wise Accuracy:
  Class 0: 0.6910
  Class 1: 0.9640
  Class 2: 0.0220
  Class 3: 0.6720
  Class 4: 0.6890
  Class 5: 0.9320
  Class 6: 0.8330
  Class 7: 0.9680
  Class 8: 0.9630
  Class 9: 0.9010

[Round 4]
Test Accuracy: 0.7941 | Loss: 0.5449
Class-wise Accuracy:
  Class 0: 0.8170
  Class 1: 0.9680
  Class 2: 0.0580
  Class 3: 0.7460
  Class 4: 0.7980
  Class 5: 0.9470
  Class 6: 0.7590
  Class 7: 0.9710
  Class 8: 0.9720
  Class 9: 0.9050

[Ro